# Computing Sentiment analysis on each tweet
----
Using the Module "TextBlob", we firstly calculate the sentiment of a given text where the value ranges form value ranges from -1 (very negative) to +1 (very positive).

In [14]:
import pandas as pd
from textblob import TextBlob
from datetime import datetime
import chardet   

# Function to calculate sentiment using TextBlob
def calculate_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

data = pd.read_csv('tweets_TRUMP_2016_2019.csv', encoding = "UTF-16")

# Convert 'created_at' to datetime
data['created_at'] = pd.to_datetime(data['created_at'], format='%m-%d-%Y %H:%M:%S')


# Calculate sentiment for each text
sentiments = []
for text in data['text']:
    sentiment = calculate_sentiment(text)
    sentiments.append(sentiment)

# Append the sentiment scores as a new column
data['sentiment'] = sentiments

data.head()


,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str,sentiment
0,Twitter for iPhone,Our prayers are with the families of the victi...,2019-12-31 00:57:13,39706,191167,False,1211813523581546496,0.600000
1,Twitter for iPhone,RT @realDonaldTrump: https://t.co/Mn4EJ7Jbh8,2019-12-31 00:00:34,40917,0,True,1211799266974478337,0.000000
2,Twitter for iPhone,RT @BreitbartNews: This Christmas let the impe...,2019-12-30 22:23:36,6012,0,True,1211774864362737665,0.000000
3,Twitter for iPhone,He is Fake News will always be Fredo to us. I ...,2019-12-30 22:23:04,20037,81302,False,1211774730258264066,-0.558333
4,Twitter for iPhone,Thank you to highly respected Jewish leader Do...,2019-12-30 14:28:26,21588,112303,False,1211655287247126535,0.386667
